In [1]:
import time
import string
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from scipy.sparse import hstack

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import average_precision_score

import keras

In [2]:
XY = pd.read_csv('XY.csv', header = 0)
XY.head(3)

,id,text,text_stemmed,text_lemmatised,normal,threat,insult,obscenity
0,41127,дворника надо тоже уничтожить!,дворник надо тоже уничтож,дворник надо тоже уничтожить,0,1,0,0
1,6812,"моя старшая неделю шипела, не принимала подкид...",моя старш недел шипел не принима подкидыш кото...,мой старший неделя шипеть не принимать подкиды...,1,0,0,0
2,6256,полностью с вами согласна!,полност с вам согласн,полностью с вы согласный,1,0,0,0


In [3]:
XY_train, XY_test = train_test_split(XY, test_size = 0.3, shuffle = True, random_state = 42)
XY_train.reset_index(drop = True, inplace = True)
XY_test.reset_index(drop = True, inplace = True)
XY_train_abn = XY_train.loc[XY_train.normal == 0, :].reset_index(drop = True)
XY_train.shape, XY_train_abn.shape

((104142, 8), (18547, 8))

In [13]:
trans = TfidfVectorizer(min_df = 3)
trans_stemmed = TfidfVectorizer(min_df = 3)
trans_lemmatised = TfidfVectorizer(min_df = 3)
X_train = trans.fit_transform(XY_train.text)
X_train_stemmed = trans_stemmed.fit_transform(XY_train.text_stemmed)
X_train_lemmatised = trans_lemmatised.fit_transform(XY_train.text_lemmatised)
X_test = trans.transform(XY_test.text)
X_test_stemmed = trans_stemmed.transform(XY_test.text_stemmed)
X_test_lemmatised = trans_lemmatised.transform(XY_test.text_lemmatised)

len(trans.vocabulary_), len(trans_stemmed.vocabulary_), len(trans_lemmatised.vocabulary_)

(36330, 21061, 21325)

In [25]:
model = keras.Sequential()
model.add(keras.layers.Dense(800, activation = 'relu', input_dim = len(trans_lemmatised.vocabulary_)))
model.add(keras.layers.Dropout(0.8))
model.add(keras.layers.Dense(4, activation = 'softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=[keras.metrics.AUC()])
model.fit(X_train_lemmatised.astype('float16'), XY_train.loc[:, 'normal':'obscenity'].values,
          epochs = 5, batch_size = 128)

Epoch 1/5
814/814 [==============================] - 43s 53ms/step - loss: 0.4326 - auc_8: 0.9722
Epoch 2/5
814/814 [==============================] - 43s 53ms/step - loss: 0.4077 - auc_8: 0.9777
Epoch 3/5
814/814 [==============================] - 43s 53ms/step - loss: 0.4401 - auc_8: 0.9769
Epoch 4/5
814/814 [==============================] - 44s 54ms/step - loss: 0.4565 - auc_8: 0.9767
Epoch 5/5
814/814 [==============================] - 44s 54ms/step - loss: 0.4835 - auc_8: 0.9755


In [26]:
average_precision_score(y_score = model.predict_proba(X_test_lemmatised.astype('float16')),
                        y_true = XY_test.loc[:, 'normal':'obscenity'].values)

0.8503708249382609

In [9]:
X_final_test = pd.read_csv('X_final_test.csv', header = 0)

In [ ]:
# trans = TfidfVectorizer(min_df = 3)
# X_train = trans.fit_transform(XY.text)
# X_test = trans.transform(X_final_test.text)